In [1]:
import logging
import sys

root = logging.getLogger()
root.setLevel(logging.INFO)

handler = logging.StreamHandler(sys.stdout)
handler.setLevel(logging.INFO)
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
handler.setFormatter(formatter)
root.addHandler(handler)

In [2]:
import sys

sys.path.append('../scripts')
sys.path.append('../')

from scripts import graph_osm_loader
from scripts import pipeline, centroids_graph_builder, clustering
from scripts.path_findings import excraction_pfa, dijkstra_pfa
from scripts import utils
import scripts

In [3]:
    # 'PARIS': 'R71525',
    # 'Prague': 'R435514',
    # 
    # 'DUBAI': 'R3766483',


In [6]:
GRAPH_ID = 'R71525'  # R13470549 R2555133 R3766483
# примеры id есть в graph_osm_loader.py
g = graph_osm_loader.get_graph(GRAPH_ID)
print(len(g.nodes), len(g.edges))

9507 14804


In [5]:
GRAPH_ID = 'R435514'  # R13470549 R2555133 R3766483
# примеры id есть в graph_osm_loader.py
g = graph_osm_loader.get_graph(GRAPH_ID)
print(len(g.nodes), len(g.edges))

21702 30009


In [29]:
GRAPH_ID = 'R3766483'  # R13470549 R2555133 R3766483
# примеры id есть в graph_osm_loader.py
g = graph_osm_loader.get_graph(GRAPH_ID)
print(len(g.nodes), len(g.edges))

59342 85597


In [7]:
points = utils.read_points(GRAPH_ID, g, num=1000)

In [8]:
nodes = g.nodes()

In [9]:
import numpy as np


def dst(da, db):
    return ((da['x'] - db['x']) ** 2 + (da['y'] - db['y']) ** 2) ** 0.5 / 360 * 2 * np.pi * 6371.01 * 1000 * 0.56


def node_dst(u, v):
    da = nodes[u]
    db = nodes[v]
    return dst(da, db)

In [10]:
from pympler.asizeof import asizeof

In [16]:

cms_resolver = clustering.LouvainCommunityResolver(resolution=2)

t, cg = centroids_graph_builder.CentroidGraphBuilder().build_with_time(g, cms_resolver)
t, asizeof(cg)/1024/1024, asizeof(g)/1024/1024

find centroids:   0%|          | 0/109 [00:00<?, ?it/s]

find edges:   0%|          | 0/109 [00:00<?, ?it/s]

(10.449612855911255, 1.374359130859375, 68.13776397705078)

In [17]:
from scripts.path_findings.arc_flags_builder import ArcFlagsBuilder
from scripts.path_findings.alt_builder import RandomAltBuilder, ClusterAltBuilder

baseline = dijkstra_pfa.Dijkstra(g)
# 
# test_alg = excraction_pfa.ExtractionPfa(
#     g = g,
#     cg=cg,
#     upper=dijkstra_pfa.Dijkstra(cg.g),
#     down=dijkstra_pfa.Dijkstra(g)
# )

test_alg = ArcFlagsBuilder().build(g, cg.cls2hubs)
# test_alg = dijkstra_pfa.AStar(
#     g = g,
#     h = node_dst
# )
# test_alg = ClusterAltBuilder(0.1).build_ch_pfa(g)

  0%|          | 0/109 [00:00<?, ?it/s]

In [18]:
stat = pipeline.PfaComparator(
    baseline=baseline,
    test_algorithm=test_alg,
    points=points,
    workers=6
).compare()

start 0


find paths:   0%|          | 0/167 [00:00<?, ?it/s]

start 1


find paths:   0%|          | 0/167 [00:00<?, ?it/s]

start 2


...

find paths:   0%|          | 0/167 [00:00<?, ?it/s]

start 4


find paths:   0%|          | 0/166 [00:00<?, ?it/s]

start 5


find paths:   0%|          | 0/166 [00:00<?, ?it/s]

In [19]:
stat


        err_max:    0.0000
        err_min:    0.0000
        err_median: 0.0000
        err_mean:   0.0000
        
        acceleration_max:    2.8983
        acceleration_min:    0.5080
        acceleration_median: 0.9242
        acceleration_mean:   0.9503
        

In [25]:
np.min([sum(d['arcs']) for u,v,d in g.edges(data=True)])

np.int64(109)

In [23]:
a = [0] * 19
a[0] = 1

In [24]:
a

[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]